Import libraries

In [1]:
import pandas as pd
from faker import Faker
import random

In [2]:
def format_harga(harga):
    return f"IDR {harga:,.0f}".replace(",", ".")

1. Buatlah sebuah daftar atau list ‘data_penjualan’ yang berisi informasi seperti 
IDproduk, nama produk, harga, jumlah terjual (quantity), dan tanggal penjualan yang 
disusun dalam struktur data dictionary.

In [3]:
def generate_data(num_days=4, products_per_day=3):
    faker = Faker()
    dates = pd.date_range("2024-08-06", periods=num_days).strftime('%Y-%m-%d')

    # Use faker to generate product names and simulate food-related names
    food_names = [f"{faker.word()} {random.choice(['Pizza', 'Burger', 'Pasta', 'Salad', 'Sushi', 'Soup', 'Steak', 'Sandwich'])}" 
                  for _ in range(products_per_day * num_days)]
    
    data_penjualan = pd.DataFrame({
        "id": ["GN" + str(random.randint(100, 999)) for _ in range(products_per_day * num_days)],
        "nama": food_names,
        "harga": [random.randint(20000, 200000) for _ in range(products_per_day * num_days)],
        "quantity": [random.randint(1, 100) for _ in range(products_per_day * num_days)],
        "tanggal": [date for date in dates for _ in range(products_per_day)]
    })

    # Apply the price formatting function
    data_penjualan['harga_formatted'] = data_penjualan['harga'].apply(format_harga)

    return data_penjualan

# Call the function to get the data
data_penjualan = generate_data()
print(data_penjualan[['id', 'nama', 'harga_formatted', 'quantity', 'tanggal']])

       id             nama harga_formatted  quantity     tanggal
0   GN999    because Salad     IDR 135.571        28  2024-08-06
1   GN913      music Salad      IDR 85.122        58  2024-08-06
2   GN193         son Soup      IDR 38.592        13  2024-08-06
3   GN326     theory Steak     IDR 105.070        19  2024-08-07
4   GN516       wind Pizza     IDR 194.309        20  2024-08-07
5   GN725       help Salad     IDR 176.399        19  2024-08-07
6   GN750  collection Soup     IDR 116.817        26  2024-08-08
7   GN702        tax Sushi     IDR 176.321         4  2024-08-08
8   GN232    beat Sandwich      IDR 96.713        58  2024-08-08
9   GN676       meet Sushi      IDR 25.534        30  2024-08-09
10  GN575       time Pasta      IDR 92.430        19  2024-08-09
11  GN962   practice Pasta     IDR 120.779        71  2024-08-09


2. Kemudian buatlah fungsi ‘hitung_pendapatan’ yang mengembalikan daftar baru dari semua informasi produk yang disertai total pendapatan untuk setiap produk.

In [4]:
def hitung_pendapatan(df):
    # Vectorized operation in Pandas for calculating revenue
    df['pendapatan'] = df['harga'] * df['quantity']
    df['pendapatan_formatted'] = df['pendapatan'].apply(lambda x: format_harga(int(x)))
    return df[['id', 'nama', 'harga_formatted', 'quantity', 'tanggal', 'pendapatan', 'pendapatan_formatted']]

data_with_pendapatan = hitung_pendapatan(data_penjualan)
print(data_with_pendapatan.to_string(index=False)) 

   id            nama harga_formatted  quantity    tanggal  pendapatan pendapatan_formatted
GN999   because Salad     IDR 135.571        28 2024-08-06     3795988        IDR 3.795.988
GN913     music Salad      IDR 85.122        58 2024-08-06     4937076        IDR 4.937.076
GN193        son Soup      IDR 38.592        13 2024-08-06      501696          IDR 501.696
GN326    theory Steak     IDR 105.070        19 2024-08-07     1996330        IDR 1.996.330
GN516      wind Pizza     IDR 194.309        20 2024-08-07     3886180        IDR 3.886.180
GN725      help Salad     IDR 176.399        19 2024-08-07     3351581        IDR 3.351.581
GN750 collection Soup     IDR 116.817        26 2024-08-08     3037242        IDR 3.037.242
GN702       tax Sushi     IDR 176.321         4 2024-08-08      705284          IDR 705.284
GN232   beat Sandwich      IDR 96.713        58 2024-08-08     5609354        IDR 5.609.354
GN676      meet Sushi      IDR 25.534        30 2024-08-09      766020          

3. Buatlah fungsi ‘average_penjualan’ yang akan mengembalikan rata-rata penjualan di tanggal tertentu sesuai inputan user. Jangan lupa buatlah exception handling apabila tanggal tidak ditemukan atau tidak sesuai format.

In [5]:
def average_penjualan(df, tanggal):
    try:
        sales_on_date = df[df['tanggal'] == tanggal]
        if sales_on_date.empty:
            raise ValueError("Tanggal tidak ditemukan.")
        
        # Compute average using pandas operations
        avg_sales = (sales_on_date['harga'] * sales_on_date['quantity']).mean()
        return format_harga(int(avg_sales))
    
    except ValueError as e:
        return str(e)
    except Exception as e:
        return "Format tanggal tidak sesuai."

# Example usage
tanggal_input = input("skibidi")
average = average_penjualan(data_penjualan, tanggal_input)
print(f"Rata-rata penjualan pada tanggal {tanggal_input}: {average}")

Rata-rata penjualan pada tanggal : Tanggal tidak ditemukan.


4. Buatlah fungsi ‘total_penjualan’ sebagai generator dengan inputan data hasil ‘hitung_pendapatan’ yang akan menghasilkan penjualan per tanggal pada setiap produk. Contoh output:

In [6]:
def total_penjualan(df):
    grouped = df.groupby('tanggal')['pendapatan'].sum().reset_index()
    for _, row in grouped.iterrows():
        yield {"tanggal": row['tanggal'], "total_penjualan": format_harga(int(row['pendapatan']))}

data_with_pendapatan = hitung_pendapatan(data_penjualan)
for penjualan in total_penjualan(data_with_pendapatan):
    print(f"Tanggal: {penjualan['tanggal']}, Total Penjualan: {penjualan['total_penjualan']}")


Tanggal: 2024-08-06, Total Penjualan: IDR 9.234.760
Tanggal: 2024-08-07, Total Penjualan: IDR 9.234.091
Tanggal: 2024-08-08, Total Penjualan: IDR 9.351.880
Tanggal: 2024-08-09, Total Penjualan: IDR 11.097.499
